In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy
import seaborn as sns

### Загрузите набор данных banknote.txt. Необходимо решить задачу классификации подлинности банкнот (предсказать переменную class).

In [ ]:
data = pd.read_csv('banknote.txt', sep='\s+')
print(data)